In [1]:
require(tidyverse)
require(data.table)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [4]:
imp_df <- fread('../meta_flipfix/imp-jobs.tsv') %>% select(phe, meta) %>%
rename('imp_meta' = 'meta')


In [32]:
imp_df %>% dim()

[1] 40  2

phe,imp_meta
<chr>,<chr>
Alanine_aminotransferase,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Alanine_aminotransferase.sumstats.tsv.gz
Albumin,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Albumin.sumstats.tsv.gz
Alkaline_phosphatase,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Alkaline_phosphatase.sumstats.tsv.gz
Apolipoprotein_A,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Apolipoprotein_A.sumstats.tsv.gz
Apolipoprotein_B_adjstatins,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Apolipoprotein_B_adjstatins.sumstats.tsv.gz
Apolipoprotein_B,/oak/stanford/groups/mrivas/projects/biomarkers/meta/plink_imputed/filtered/GLOBAL_Apolipoprotein_B.sumstats.tsv.gz


In [39]:
read_array_df <- function(){
    array_df <- fread('../meta_flipfix/jobs.lst', header=F)
    colnames(array_df) <- c('array_meta')    

    array_df %>%
    mutate(
        phe = str_replace(str_replace(basename(array_meta), '^METAANALYSIS_', ''), '_1.tbl$', ''), 
        array_meta = str_replace(str_replace(array_meta, '/meta/', '/meta_flipfixed/'), '.tbl$', '.tbl.gz')
    )    
}

In [40]:
array_df <- read_array_df()

In [48]:
all_df <- array_df %>%
inner_join(
    imp_df, by='phe'
) %>%
select(phe, array_meta, imp_meta)

In [50]:
all_df %>% 
fwrite('cascade.input.files.tsv', sep='\t')


# phenotype name mismatch

In [42]:
array_phes <- array_df %>% select(phe) %>% pull()

In [37]:
imp_phes <- imp_df %>% select(phe) %>% pull()

In [46]:
imp_phes[! imp_phes %in% array_phes]

[1] "Apolipoprotein_B_adjstatins" "Cholesterol_adjstatins"     
[3] "Cholesterol"                 "LDL_direct_adjstatins"      
[5] "LDL_direct"

In [47]:
array_phes[! array_phes %in% imp_phes]

[1] "Apolipoprotein_B_adjust_statins" "AST_ALT_ratio"                  
[3] "Cholesterol_adjust_statins"      "LDL_direct_adjust_statins"